In [2]:

!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install transformers==4.37.2
!pip install datasets==2.17.0
!pip install evaluate==0.4.1
!pip install rouge-score==0.1.2

In [3]:
import matplotlib.pyplot as plt
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize

import torch

from tqdm import tqdm

from datasets import load_dataset, load_from_disk, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline, set_seed


c:\Users\uurce\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import huggingface_hub

huggingface_hub.login(token="TOKEN_API")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.


ValueError: Invalid token passed!

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "facebook/bart-large-xsum"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

### Load Dataset

In [7]:
import requests
import zipfile
import os

# Define the URL of the ZIP file
url = 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip'

# Define the file paths
zip_file_path = 'summarizer-data.zip'
extracted_folder_path = 'summarizer-data'

# Download the ZIP file
response = requests.get(url)
with open(zip_file_path, 'wb') as zip_file:
    zip_file.write(response.content)

# Extract the contents of the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# Remove the ZIP file
os.remove(zip_file_path)

print('Downloaded and extracted the ZIP file successfully.')


Downloaded and extracted the ZIP file successfully.


In [8]:
dataset = load_from_disk("summarizer-data/samsum_dataset/")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
# Calculate split lengths
split_lengths = [len(dataset[split]) for split in dataset]


print("Split lengths:", split_lengths)


print("Features:", dataset['train'].column_names)


print("\nDialogue:")
print(dataset["test"][1]["dialogue"])


print("\nSummary:")
print(dataset["test"][1]["summary"])


Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [12]:
train_df = pd.DataFrame(dataset["train"])
test_df = pd.DataFrame(dataset["test"])
validation_df = pd.DataFrame(dataset["validation"])
